**Selected Dataset:**  *Mach München Besser – Open311 GeoReport v2* dataset,  
published by the **City of Munich (Landeshauptstadt München)**.  
It contains anonymized **citizen complaints and issue reports** submitted via the city’s official Open311 API,  
including unstructured text fields (`description`) and structured metadata (`service_name`, `status`, `requested_datetime`, etc.).  

**Source:** https://machmuenchenbesser.de/georeport/v2/requests.json 

### (1) Loading Dataset & Displaying Unique Columns

In [4]:
import requests, pandas as pd
from datetime import datetime, timedelta

url = "https://machmuenchenbesser.de/georeport/v2/requests.json"
end = datetime.now()
start = end - timedelta(days=60)

params = {
    "start_date": start.strftime("%Y-%m-%dT%H:%M:%SZ"),
    "end_date": end.strftime("%Y-%m-%dT%H:%M:%SZ")
}

r = requests.get(url, params=params, timeout=60)
r.raise_for_status()
df = pd.DataFrame(r.json())

print("Columns in dataset:")
print(df.columns.tolist(), "\n")

print("Example rows:")
display(df.head(5))

Columns in dataset:
['service_request_id', 'title', 'description', 'lat', 'long', 'address_string', 'service_name', 'requested_datetime', 'updated_datetime', 'status', 'media_url', 'status_note', 'service_code'] 

Example rows:


,service_request_id,title,description,lat,long,address_string,service_name,requested_datetime,updated_datetime,status,media_url,status_note,service_code
0,63656-2025,#63656-2025 Leuchte ausgefallen,Der Lichtmast funktioniert schon etwas länger ...,48.088190,11.489650,"81476 München, Lechbrucker Straße 19",Leuchte ausgefallen,2025-10-10T19:57:13+02:00,2025-10-10T20:10:01+02:00,open,,,24.17
1,63660-2025,#63660-2025 Leuchte ausgefallen,Mast Nr. 3 komplett dunkel,48.166289,11.590986,"80802 München, Dietlindenstraße 4",Leuchte ausgefallen,2025-10-11T01:16:13+02:00,2025-10-11T01:40:04+02:00,open,http://machmuenchenbesser.de/sites/default/fil...,,24.17
2,63671-2025,#63671-2025 Plakat befindet sich in Sperrgebie...,None,48.152847,11.533646,"80634 München, Nymphenburger Straße 160",Plakat befindet sich in Sperrgebiet/Bushaltest...,2025-10-11T10:34:21+02:00,2025-10-20T15:53:59+02:00,open,http://machmuenchenbesser.de/sites/default/fil...,,None
3,63682-2025,#63682-2025 Plakat befindet sich in Sperrgebie...,None,48.083871,11.517617,"81479 München, Aidenbachstraße 223",Plakat befindet sich in Sperrgebiet/Bushaltest...,2025-10-11T13:13:20+02:00,2025-12-01T13:20:16+01:00,open,http://machmuenchenbesser.de/sites/default/fil...,,None
4,63688-2025,#63688-2025 Plakat blockiert Sicht/Straße/Geh-...,None,48.118509,11.574797,"81543 München, Kupferhammerstraße 2",Plakat blockiert Sicht/Straße/Geh-/Radweg,2025-10-11T14:16:06+02:00,2025-12-08T11:53:08+01:00,closed,http://machmuenchenbesser.de/sites/default/fil...,,None


### (2) Dataset Sampling

This dataset consolidates **unresolved (open) and resolved (closed) citizen complaints** submitted via the City of Munich’s *Mach München Besser* Open311 portal within the period **2020-01-01 to 2025-12-01**.  
Its purpose is to provide a **time-bounded corpus of open complaints** for NLP analysis (e.g., topic extraction) focused on the most pressing, still-unresolved issues.  
To keep the analysis both meaningful and manageable, over **900 complaints** are retained.

In [5]:
# Munich Open311 – fetch OPEN and CLOSED separately, then merge + dedupe, then drop empty descriptions
import os, time, requests, pandas as pd
from datetime import datetime, timedelta, timezone
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

BASE_URL = "https://machmuenchenbesser.de/georeport/v2/requests.json"

START_ISO = "2020-01-01T00:00:00Z"   # inclusive
END_ISO   = "2025-12-01T00:00:00Z"   # exclusive
KEEP_COLS = ["service_request_id","service_name","title","description","requested_datetime","status"]

# Heuristics
MAX_WINDOW_DAYS  = 90
MIN_WINDOW_DAYS  = 7
CAP_SUSPECT      = 100
REQUEST_SLEEP    = 0.12
TIMEOUT_SEC      = 45

def to_utc(s: str) -> datetime:
    return datetime.fromisoformat(s.replace("Z","+00:00")).astimezone(timezone.utc)

def iso(dt: datetime) -> str:
    return dt.astimezone(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

def make_session() -> requests.Session:
    s = requests.Session()
    retries = Retry(total=4, backoff_factor=0.6,
                    status_forcelist=(429,500,502,503,504),
                    allowed_methods=frozenset(["GET"]))
    ad = HTTPAdapter(max_retries=retries, pool_connections=8, pool_maxsize=8)
    s.mount("http://", ad); s.mount("https://", ad)
    s.headers.update({"Accept":"application/json","User-Agent":"munich-open311-nlp/fetch-split/1.0"})
    return s

def fetch_slice(session: requests.Session, start_dt: datetime, end_dt: datetime, status: str) -> list[dict]:
    params = {"start_date": iso(start_dt), "end_date": iso(end_dt), "status": status}
    r = session.get(BASE_URL, params=params, timeout=TIMEOUT_SEC)
    r.raise_for_status()
    data = r.json()
    if isinstance(data, list): return data
    if isinstance(data, dict):
        for v in data.values():
            if isinstance(v, list): return v
    return []

def collect_window(session: requests.Session, start_dt: datetime, end_dt: datetime, status: str) -> list[dict]:
    span_days = max(int((end_dt - start_dt).total_seconds() // 86400), 0)
    if span_days > MAX_WINDOW_DAYS:
        mid = start_dt + timedelta(days=MAX_WINDOW_DAYS)
        return (collect_window(session, start_dt, mid, status)
                + collect_window(session, mid, end_dt, status))

    rows = fetch_slice(session, start_dt, end_dt, status)
    print(f"  [{status:6}] fetched {len(rows):3d} ({span_days:2d}d): {iso(start_dt)} → {iso(end_dt)}")
    time.sleep(REQUEST_SLEEP)

    if len(rows) >= CAP_SUSPECT and span_days > MIN_WINDOW_DAYS:
        half = (end_dt - start_dt) / 2
        mid = start_dt + half
        # guard against degenerate midpoint
        if mid <= start_dt + timedelta(seconds=1) or mid >= end_dt - timedelta(seconds=1):
            shrink = timedelta(days=min(MIN_WINDOW_DAYS, span_days//2 or 1))
            mid = min(start_dt + shrink, end_dt - timedelta(seconds=1))
        left  = collect_window(session, start_dt, mid, status)
        right = collect_window(session, mid, end_dt, status)
        return left + right

    return rows

def build_year_windows(start_iso: str, end_iso: str) -> list[tuple[datetime, datetime]]:
    s, e = to_utc(start_iso), to_utc(end_iso)
    out, cur = [], s
    while cur < e:
        year_end = datetime(cur.year + 1, 1, 1, tzinfo=timezone.utc)
        nxt = min(year_end, e)
        out.append((cur, nxt))
        cur = nxt
    return out

def collect_status(status: str) -> pd.DataFrame:
    session = make_session()
    outer = build_year_windows(START_ISO, END_ISO)
    print(f"\nCollecting {len(outer)} year windows | status={status}")
    all_rows = []
    for i, (ws, we) in enumerate(outer, 1):
        print(f"=== Window {i}/{len(outer)}: {iso(ws)} → {iso(we)} ===")
        all_rows += collect_window(session, ws, we, status)
    df = pd.DataFrame(all_rows)
    if df.empty:
        return df
    # Deduplicate by ID, keep latest by requested_datetime
    df = df.sort_values("requested_datetime", kind="stable").drop_duplicates("service_request_id", keep="last")
    # Normalize and keep requested status only (defensive)
    df["status_norm"] = df["status"].astype(str).str.lower().str.strip()
    df = df[df["status_norm"] == status.lower()].drop(columns=["status_norm"])
    return df[KEEP_COLS].reset_index(drop=True)

# RUN: fetch separately, then merge
df_open   = collect_status("open")
df_closed = collect_status("closed")

df_all = pd.concat([df_open, df_closed], ignore_index=True)
before = len(df_all)
df_all = df_all.sort_values("requested_datetime", kind="stable").drop_duplicates("service_request_id", keep="last")
print(f"\nMerge+dedup: {before} → {len(df_all)} unique complaints")
print("Split:", {"open": len(df_open), "closed": len(df_closed)})

# Drop rows with empty descriptions 
desc_before = len(df_all)

s = df_all["description"]                # original series
mask_nonempty = s.notna()                # keep only real non-NaN
s_norm = (
    s[mask_nonempty]
      .astype(str)
      .str.replace(u"\xa0", " ", regex=False)   # remove NBSP
      .str.replace(r"\s+", " ", regex=True)     # collapse whitespace
      .str.strip()
)

# valid text = not empty and not a placeholder like "nan"/"none"
valid = s_norm.ne("") & ~s_norm.str.lower().isin({"nan", "none", "null"})

# build final mask over original index
final_mask = pd.Series(False, index=df_all.index)
final_mask.loc[s_norm.index] = valid

df_all = df_all[final_mask].reset_index(drop=True)
print(f"Dropped rows with empty/placeholder descriptions: {desc_before - len(df_all)} (kept {len(df_all)})")

# Save (no-empty-descriptions version)
out_dir = "/Users/dd/PycharmProjects/complaints_analysis/datasets/raw"
os.makedirs(out_dir, exist_ok=True)
fname = f"munich_open311_{START_ISO[:10]}_to_{END_ISO[:10]}.csv"
path = os.path.join(out_dir, fname)
df_all.to_csv(path, index=False, encoding="utf-8")
print(f"Saved {len(df_all)} rows → {path}")


=== Window 1/6: 2020-01-01T00:00:00Z → 2021-01-01T00:00:00Z ===
  [open  ] fetched   0 (90d): 2020-01-01T00:00:00Z → 2020-03-31T00:00:00Z
  [open  ] fetched   0 (90d): 2020-03-31T00:00:00Z → 2020-06-29T00:00:00Z
  [open  ] fetched   0 (90d): 2020-06-29T00:00:00Z → 2020-09-27T00:00:00Z
  [open  ] fetched   0 (90d): 2020-09-27T00:00:00Z → 2020-12-26T00:00:00Z
  [open  ] fetched   0 ( 6d): 2020-12-26T00:00:00Z → 2021-01-01T00:00:00Z
=== Window 2/6: 2021-01-01T00:00:00Z → 2022-01-01T00:00:00Z ===
  [open  ] fetched   0 (90d): 2021-01-01T00:00:00Z → 2021-04-01T00:00:00Z
  [open  ] fetched   0 (90d): 2021-04-01T00:00:00Z → 2021-06-30T00:00:00Z
  [open  ] fetched   1 (90d): 2021-06-30T00:00:00Z → 2021-09-28T00:00:00Z
  [open  ] fetched   1 (90d): 2021-09-28T00:00:00Z → 2021-12-27T00:00:00Z
  [open  ] fetched   0 ( 5d): 2021-12-27T00:00:00Z → 2022-01-01T00:00:00Z
=== Window 3/6: 2022-01-01T00:00:00Z → 2023-01-01T00:00:00Z ===
  [open  ] fetched   2 (90d): 2022-01-01T00:00:00Z → 2022-04-01T00:

In [2]:
# Displaying unqiue 'service name' complaint categories

import pandas as pd

path = "/Users/dd/PycharmProjects/complaints_analysis/datasets/raw/munich_open311_2020-01-01_to_2025-12-01.csv"
df = pd.read_csv(path, encoding="utf-8")

unique_services = df["service_name"].dropna().unique()

print(f"Total unique service_name categories: {len(unique_services)}\n")
for s in sorted(unique_services):
    print(s)

Total unique service_name categories: 27

Ampel ausgefallen
Ampel beschädigt
Ampel verschmutzt
Bank beschädigt
Brunnen beschädigt
Brunnen verschmutzt
Denkmal beschädigt
Denkmal verschmutzt
Geh- und Radweg verschmutzt
Grünstreifen verschmutzt
Hundekotbeutelspender leer
Leuchte ausgefallen
Leuchte beschädigt
Leuchte verschmutzt
Licht in ganzer Straße ausgefallen
Mülleimer überfüllt
Oktoberfest
Park beschädigt
Park verschmutzt
Plakat beschädigt
Plakat blockiert Sicht/Straße/Geh-/Radweg
Plakat noch nicht abgebaut
Spielplatz beschädigt
Spielplatz verschmutzt
Straße verschmutzt
Verkehrsschild beschädigt
Verkehrsschild verschmutzt


In [3]:
import pandas as pd

# 1. Load dataset
path = "/Users/dd/PycharmProjects/complaints_analysis/datasets/raw/munich_open311_2020-01-01_to_2025-12-01.csv"
df = pd.read_csv(path)

# 2. Clean description field (remove empty / NaN / placeholder values)
df["description"] = df["description"].astype(str).str.strip()
df = df[
    df["description"].ne("") &
    df["description"].str.lower().ne("none") &
    df["description"].str.lower().ne("nan")
]

# 3. Keep only needed columns
df = df[["service_name", "description"]].copy()

# 4. Create DataFrame with up to 2 example complaints per service_name
examples_df = (
    df.groupby("service_name", group_keys=False)
      .apply(lambda g: g.head(2))
      .reset_index(drop=True)
)
examples_df

/var/folders/_5/ns_prwp1673_94nw9yv3t7pw0000gn/T/ipykernel_57327/2538106523.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.head(2))


,service_name,description
0,Ampel ausgefallen,"Die Fußgänger Ampel an der Ecke, Schleißheimer..."
1,Ampel ausgefallen,Ampelanlage Sigl-/Zschokkestraße ausgefallen
2,Ampel beschädigt,Ampel beschädigt
3,Ampel beschädigt,Schaltkasten defekt und offen. Sehr gefährlich...
4,Ampel verschmutzt,Die gesamten Masten der LZA sind mit Werbung f...
5,Bank beschädigt,Bei 4 von 5 Bänken fehlen die Rückenlehnen.\r\...
6,Bank beschädigt,Befestigung Sitzbank Schrauben lose. Sitzpanel...
7,Brunnen beschädigt,Die Brunnen im Westpark laufen nicht. Können S...
8,Brunnen beschädigt,"Hallo! Sehr geehrte Damen und Herren,\r\nleide..."
9,Brunnen verschmutzt,Die Kategorie stimmt nicht ganz: Nicht der Bru...


From the *Mach München Besser – Open311 GeoReport v2* dataset,  
only the columns relevant to text-based topic extraction and contextual interpretation are retained.

| Column | Purpose | Justification |
|:--|:--|:--|
| **`description`** | Main complaint text | Contains the citizens’ **unstructured free-text complaints**, forming the **core input** for NLP preprocessing and topic modeling (e.g., LDA, NMF, BERTopic). |
| **`service_name`** | Official complaint category | Provides the **city-assigned category label** (e.g., “Leuchte ausgefallen”). Used later for **validating and comparing** extracted topics against official classifications. |
| **`title`** | Official complaint category with the respective complaint ID | Provides the **city-assigned category label** (e.g., “Leuchte ausgefallen”). 
| **`requested_datetime`** | Submission timestamp | Enables **temporal trend analysis** (e.g., frequency of certain issues over time) and filtering by reporting period. |
| **`status`** | Complaint state | Distinguishes **open** vs. **closed** cases, supporting additional analysis on unresolved topics. |
| **`service_request_id`** | Unique identifier | Maintains a consistent reference for each record when filtering, indexing, or merging results. |

All other fields (e.g., `lat`, `long`, `address_string`, `media_url`, `service_code`, `status_note`)  
are excluded since they do not contribute to **textual or semantic analysis** and add unnecessary complexity.

In [10]:
import pandas as pd

path = "/Users/dd/PycharmProjects/complaints_analysis/datasets/raw/munich_open311_2020-01-01_to_2025-12-01.csv"
df = pd.read_csv(path)

# Check for duplicated complaint IDs
dupe_mask = df["service_request_id"].duplicated(keep=False)
num_dupes = dupe_mask.sum()

print(f"Total duplicates found: {num_dupes}")

Total duplicates found: 0


In [12]:
import pandas as pd

# Load your dataset (adjust path as needed)
path = "/Users/dd/PycharmProjects/complaints_analysis/datasets/raw/munich_open311_2020-01-01_to_2025-12-01.csv"
df = pd.read_csv(path)

# Identify empty or missing descriptions
empty_mask = df["description"].isna() | df["description"].astype(str).str.strip().eq("")

# Summary stats
total_rows = len(df)
empty_count = empty_mask.sum()
percent_empty = (empty_count / total_rows) * 100

print(f"🧾 Total complaints: {total_rows}")
print(f"⚠️ Empty descriptions: {empty_count} ({percent_empty:.2f}%)")

# Optional: show some exampless
if empty_count > 0:
    print("\nExamples of rows with empty descriptions:")
    display(df.loc[empty_mask, ["service_request_id", "service_name", "title", "status"]].head(10))

🧾 Total complaints: 955
⚠️ Empty descriptions: 0 (0.00%)
